# Table of Contents
0. Notebook Setup
1. SQL
2. ProductDF
3. TimeDF
4. Exporting Dataframes

## 0. Notebook Setup

In [365]:
# 0.0 -- import packages
import pyodbc
import pandas as pd
import numpy as np
import regex as re
import os
from pathlib import Path

In [366]:
# 0.1 -- Check pyodbc drivers
for driver in pyodbc.drivers():
    print(driver)

SQL Server
PostgreSQL ANSI(x64)
PostgreSQL Unicode(x64)
MySQL ODBC 8.0 ANSI Driver
MySQL ODBC 8.0 Unicode Driver
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [367]:
# 0.2 -- Define our server name and database name
# And establish a connection to the database
SERVERNAME = os.environ.get('SQL_EXPRESS_SERVER')
server = SERVERNAME + "\SQLEXPRESS"
database = "WideWorldImporters"
cnxn = pyodbc.connect(f"""
DRIVER={{ODBC Driver 17 for SQL SERVER}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
MARS_CONNECTION=yes;
""")

In [368]:
# 0.3 -- Create the connection cursor
cursor = cnxn.cursor()

## 1. SQL


Logic of table joins used in the following query to create **'productdf'**

| Table | Alias | Reason | Key |
| :--------- | :--------- | :--------- | :--------- |
| 1. Sales.InvoiceLines | line | Numerical sales data | StockItemID, InvoiceID
| 2. Warehouse.StockItems | s | StockItemID/Name | StockItemID, ColorID
| 3. Sales.Invoices | inv | InvoiceID | InvoiceID, OrderID
| 4. Sales.Orders | o | Chronological data | OrderID 
| 5. Warehouse.Colors | c | Color | ColorID
| 6. Warehouse.StockItemHoldings | h | LastCostPrice | StockItemID


Notes
- LineProfit = UnitPrice - LastCostPrice * Quantity

In [369]:
# 1.0 -- Query to create our productdf dataframe
# We'd like to see a breakdown of sales success per category>subcategory>product>size/color
query = """SELECT 
            YEAR(o.OrderDate) AS Year,
            s.StockItemName, 
            s.Size, 
            c.ColorName AS Color, 
            SUM(line.Quantity) AS Quantity, 
            line.UnitPrice, 
            h.LastCostPrice,
            line.UnitPrice-h.LastCostPrice AS UnitMargin, 
            (line.UnitPrice-h.LastCostPrice) / line.UnitPrice AS PctMargin,
            SUM(line.Quantity*line.UnitPrice) AS TotalUnitPrice, 
            SUM(line.LineProfit) AS LineProfit
        FROM Sales.InvoiceLines line 
        JOIN Warehouse.StockItems s
            ON line.StockItemID = s.StockItemID 
        JOIN Sales.Invoices inv
            ON inv.InvoiceID = line.InvoiceID
        JOIN Sales.Orders o 
            ON o.OrderID = inv.OrderID 
        LEFT JOIN Warehouse.Colors c
            ON c.ColorID = s.ColorID 
        JOIN Warehouse.StockItemHoldings h
            ON h.StockItemID = line.StockItemID
        GROUP BY 
            YEAR(o.OrderDate), 
            s.StockItemName, 
            s.Size, 
            c.ColorName, 
            line.UnitPrice, 
            h.LastCostPrice
        ORDER BY 
            YEAR(o.OrderDate), 
            s.StockItemName"""
productdf = pd.read_sql(query, cnxn) 
print(productdf.shape)
display(productdf.head(5))

(912, 11)


,Year,StockItemName,Size,Color,Quantity,UnitPrice,LastCostPrice,UnitMargin,PctMargin,TotalUnitPrice,LineProfit
0,2013,"""The Gu"" red shirt XML tag t-shirt (Black) 3XL",3XL,Black,18312,18.0,8.0,10.0,0.555556,329616.0,183120.0
1,2013,"""The Gu"" red shirt XML tag t-shirt (Black) 3XS",3XS,Black,16692,18.0,7.0,11.0,0.611111,300456.0,183612.0
2,2013,"""The Gu"" red shirt XML tag t-shirt (Black) 4XL",4XL,Black,8448,18.0,8.0,10.0,0.555556,152064.0,84480.0
3,2013,"""The Gu"" red shirt XML tag t-shirt (Black) 5XL",5XL,Black,16764,18.0,8.0,10.0,0.555556,301752.0,167640.0
4,2013,"""The Gu"" red shirt XML tag t-shirt (Black) 6XL",6XL,Black,17964,18.0,8.0,10.0,0.555556,323352.0,179640.0


Logic of table joins used in the following query to create **'timedf'**

| Table | Alias | Reason | Key |
| :--------- | :--------- | :--------- | :--------- |
| 1. Sales.InvoiceLines | line | Numerical sales data | StockItemID, InvoiceID
| 2. Warehouse.StockItems | s | StockItemID/Name | StockItemID
| 3. Sales.Invoices | inv | InvoiceID | InvoiceID, OrderID
| 4. Sales.Orders | o | Chronological data | OrderID 

Notes
- LineProfit = UnitPrice - LastCostPrice * Quantity

In [370]:
# 1.0 -- Query to create our timedf dataframe
# We're interested in comparing transformations of revenue and profit over time
query = """SELECT 
            YEAR(o.OrderDate) AS Year, 
            MONTH(o.OrderDate) AS Month,
            s.StockItemName, 
            SUM(line.Quantity) AS Quantity, 
            SUM(line.Quantity*line.UnitPrice) AS TotalUnitPrice, 
            SUM(line.Quantity*line.UnitPrice) - SUM(line.LineProfit) AS Revenue,
            SUM(line.LineProfit) AS LineProfit
        FROM Sales.InvoiceLines line 
        JOIN Warehouse.StockItems s
            ON line.StockItemID = s.StockItemID 
        JOIN Sales.Invoices inv
            ON inv.InvoiceID = line.InvoiceID
        JOIN Sales.Orders o 
            ON o.OrderID = inv.OrderID 
        GROUP BY YEAR
            (o.OrderDate), 
            MONTH(o.OrderDate), 
            s.StockItemName
        ORDER BY 
            YEAR(o.OrderDate), 
            MONTH(o.OrderDate), 
            s.StockItemName"""
timedf = pd.read_sql(query, cnxn) 
print(timedf.shape)
display(timedf.head(5))

(9019, 7)


,Year,Month,StockItemName,Quantity,TotalUnitPrice,Revenue,LineProfit
0,2013,1,"""The Gu"" red shirt XML tag t-shirt (Black) 3XL",1512,27216.0,12096.0,15120.0
1,2013,1,"""The Gu"" red shirt XML tag t-shirt (Black) 3XS",1380,24840.0,9660.0,15180.0
2,2013,1,"""The Gu"" red shirt XML tag t-shirt (Black) 4XL",1236,22248.0,9888.0,12360.0
3,2013,1,"""The Gu"" red shirt XML tag t-shirt (Black) 5XL",1596,28728.0,12768.0,15960.0
4,2013,1,"""The Gu"" red shirt XML tag t-shirt (Black) 6XL",1476,26568.0,11808.0,14760.0


In [371]:
# 1.1 -- Checking in on our NaN count
# NaNs in Size and Color are acceptable - not every item has a specific attribute of these types
def nanpct(df) :
    print(df.shape)
    dfnan = df.isna().sum()
    dfnanpct = round(df.isna().sum() / len(df) * 100, 1)     
    display(pd.DataFrame(pd.concat((dfnan, dfnanpct), axis = 1).rename(columns= {0: 'NaNs', 1: '%total'})))
print('---productdf---')
nanpct(productdf)
print('---timedf---')
nanpct(timedf)

---productdf---
(912, 11)


,NaNs,%total
Year,0,0.0
StockItemName,0,0.0
Size,284,31.1
Color,398,43.6
Quantity,0,0.0
UnitPrice,0,0.0
LastCostPrice,0,0.0
UnitMargin,0,0.0
PctMargin,0,0.0
TotalUnitPrice,0,0.0


---timedf---
(9019, 7)


,NaNs,%total
Year,0,0.0
Month,0,0.0
StockItemName,0,0.0
Quantity,0,0.0
TotalUnitPrice,0,0.0
Revenue,0,0.0
LineProfit,0,0.0


## 2. ProductDF

In [372]:
# 2.0 -- Next, we'll use regex to remove end-of-string size and color details from StockItemName values in productdf
# It's preferred to have these details isolated in their respective columns and avoid redundancy
# e.g. Before regex:
display(productdf.iloc[[20, 40, 60, 80, 100, 120],1:4])

,StockItemName,Size,Color
20,"""The Gu"" red shirt XML tag t-shirt (White) M",M,White
40,32 mm Anti static bubble wrap (Blue) 20m,20m,Blue
60,Black and orange glass with care despatch tape...,48mmx75m,None
80,DBA joke mug - SELECT caffeine FROM mug (Black),None,Black
100,Developer joke mug - there are 10 types of peo...,None,Black
120,Furry gorilla with big eyes slippers (Black) XL,XL,Black


In [373]:
# 2.1 -- Using three lines of regex, making sure we're not capturing anything we don't want to
# Line 1 removes most size measurements
# Line 2 removes remaining fractional "scale" measurements
# Line 3 removes color in (Color) format
productdf['StockItemName'] = productdf['StockItemName'].apply(lambda x: re.sub(r"\s*\d*[mx]*(\d*)[mx]*(\d*)[\sx](\d*\.{,2}\d*[xsmlXSML]+)(\w{,1})$", '', str(x)))
productdf['StockItemName'] = productdf['StockItemName'].apply(lambda x: re.sub(r"\s\d+\/\d+\s*scale$", '', str(x)))
productdf['StockItemName'] = productdf['StockItemName'].apply(lambda x: re.sub(r"\s\([A-Z]\w+\)$", '', str(x)))

In [374]:
# 2.2 -- Convering StockItemName values to title case
productdf['StockItemName'] = productdf['StockItemName'].str.title()

In [375]:
# 2.3 -- The same examples from before
display(productdf.iloc[[20, 40, 60, 80, 100, 120],1:4])

,StockItemName,Size,Color
20,"""The Gu"" Red Shirt Xml Tag T-Shirt",M,White
40,32 Mm Anti Static Bubble Wrap,20m,Blue
60,Black And Orange Glass With Care Despatch Tape,48mmx75m,None
80,Dba Joke Mug - Select Caffeine From Mug,None,Black
100,Developer Joke Mug - There Are 10 Types Of Peo...,None,Black
120,Furry Gorilla With Big Eyes Slippers,XL,Black


In [376]:
# 2.4 -- Checkpoint: here's our completed productdf
productdf.head()

,Year,StockItemName,Size,Color,Quantity,UnitPrice,LastCostPrice,UnitMargin,PctMargin,TotalUnitPrice,LineProfit
0,2013,"""The Gu"" Red Shirt Xml Tag T-Shirt",3XL,Black,18312,18.0,8.0,10.0,0.555556,329616.0,183120.0
1,2013,"""The Gu"" Red Shirt Xml Tag T-Shirt",3XS,Black,16692,18.0,7.0,11.0,0.611111,300456.0,183612.0
2,2013,"""The Gu"" Red Shirt Xml Tag T-Shirt",4XL,Black,8448,18.0,8.0,10.0,0.555556,152064.0,84480.0
3,2013,"""The Gu"" Red Shirt Xml Tag T-Shirt",5XL,Black,16764,18.0,8.0,10.0,0.555556,301752.0,167640.0
4,2013,"""The Gu"" Red Shirt Xml Tag T-Shirt",6XL,Black,17964,18.0,8.0,10.0,0.555556,323352.0,179640.0


## 3. TimeDF

In [377]:
# 3.0 -- If we join our two dataframes in Tableau as they are now, we'll accumulate duplicate measurements
# Rather than dealing with them in Tableau, we'll address them here by categorizing our stock items
# Plus, we don't actually need timedf at the granularity level of stock items
# Our first step in converting our item names to categories is standardizing strings as lowercase
timedf.StockItemName = timedf.StockItemName.str.lower()

In [378]:
# 3.1 -- We're now defining a new function to sort StockItemName into categories via list comprehension
# >> If an item name contains any [list values], return the appropriate category name
def categorize_item(item_name):
    packaging = ['packaging', 'bubble', 'tape', 'box', 'bag', 'shipping', 'marker', 'cushion', 'blade', 'knife']
    clothing = ['shirt', 'furry', 'slippers', 'mask', 'hoodie']
    usb_devices = ['usb']
    toys = ['superhero', 'ride', 'remote', 'action', 'cube']

    if any([(i) in item_name for i in packaging]):
        return "Packaging"
    elif any([(i) in item_name for i in clothing]):
        return "Clothing"
    elif any([(i) in item_name for i in usb_devices]):
        return "USB Devices"
    elif any([(i) in item_name for i in toys]):
        return "Toys"
    elif 'chocolate' in item_name:
        return "Candy"
    elif 'mug' in item_name :
        return "Mugs" 
timedf.StockItemName = timedf.StockItemName.apply(categorize_item)

In [379]:
# 3.2 -- Now to rename StockItemName as Category
timedf.rename(columns={'StockItemName' : 'Category'}, inplace= True)

In [380]:
# 3.3 -- All set. Here's our new Category column
timedf.Category.value_counts()

Packaging      2747
Clothing       2583
Mugs           1722
Toys           1353
USB Devices     574
Candy            40
Name: Category, dtype: int64

In [381]:
# 3.4 -- Our next step is to combine timedf.Year and timedf.Month into a single OrderDate column
# Then, drop Year and Month
timedf['OrderDate'] = timedf[['Year', 'Month']].apply(lambda x: '-'.join(x.values.astype(str)), axis='columns')
timedf.drop(columns = ['Year', 'Month'], inplace= True)

In [382]:
# 3.5 -- And reformat OrderDate to datetime
timedf['OrderDate'] = pd.to_datetime(timedf['OrderDate'])

In [383]:
# 3.6 -- Reordering our new set of columns
timedf = timedf[['OrderDate', 'Category', 'Quantity', 'TotalUnitPrice', 'LineProfit',]]

In [384]:
# 3.7 -- This is the current state of our timedf dataframe
# We can aggregate these measurements to shrink our dataframe and expedite processes once we're in Tableau
display(timedf)

,OrderDate,Category,Quantity,TotalUnitPrice,LineProfit
0,2013-01-01,Clothing,1512,27216.0,15120.0
1,2013-01-01,Clothing,1380,24840.0,15180.0
2,2013-01-01,Clothing,1236,22248.0,12360.0
3,2013-01-01,Clothing,1596,28728.0,15960.0
4,2013-01-01,Clothing,1476,26568.0,14760.0
...,...,...,...,...,...
9014,2016-05-01,Packaging,2200,55000.0,19800.0
9015,2016-05-01,Packaging,1450,54375.0,28275.0
9016,2016-05-01,Packaging,2070,103500.0,49680.0
9017,2016-05-01,Candy,3576,30574.8,13588.8


In [389]:
# 3.8 -- Using .groupby to aggregate measurements along dimensions
# timedf was previously 9019 rows, and is now 210
timedf = timedf.groupby(['OrderDate', 'Category']).agg(sum)
display(timedf.head(15))
timedf.shape

Quantity  TotalUnitPrice  LineProfit
OrderDate  Category                                         
2013-01-01 Clothing        57285       991644.00   494549.00
           Mugs             6102        79326.00    51867.00
           Packaging      124712      2224844.85  1137180.30
           Toys             5450       401344.00   159640.00
           USB Devices      1846       111484.00    70035.50
2013-02-01 Clothing        39780       697491.00   345016.00
           Mugs             4167        54171.00    35419.50
           Packaging       93210      1655765.20   838929.30
           Toys             4037       307289.00   123226.00
           USB Devices      1232        71358.00    44812.50
2013-03-01 Clothing        64436      1121158.00   543429.50
           Mugs             5635        73255.00    47897.50
           Packaging      133267      2203427.10  1133428.45
           Toys             5453       399193.00   160013.50
           USB Devices      1875       120877.00    75966.50

(210, 3)

## 4. Exporting Dataframes

In [ ]:
# 4.0 -- Our dataframes are now ready for export
productdf.to_csv(Path('Tableau Inputs') / 'WWI_Productdf.csv')
timedf.to_csv(Path('Tableau Inputs') / 'WWI_Timedf.csv')